In [1]:
import yaml
import logging
from prefect import flow, task
from typing import Union, Dict

In [2]:
class Singleton_Echoflow:
    _instance: "Singleton_Echoflow" = None
    logger: logging.Logger = None

    def __new__(
        cls,
        log_file: Union[Dict[str, str], str] = None,
    ) -> "Singleton_Echoflow":
        
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            if log_file is not None:
                cls._instance.logger = cls._instance.logger_init(log_file)
        return cls._instance

    @classmethod
    def get_instance(self) -> "Singleton_Echoflow":
        return self._instance
    
    @classmethod
    def get_logger(self) -> "logging.Logger":
        return self.logger

    def logger_init(self, log_file: Union[Dict[str, str], str]) -> None:
        if type(log_file) == str:
            with open(log_file, "r") as file:
                logging_config_dict = yaml.safe_load(file.read())
                logging.config.dictConfig(logging_config_dict)
        else:
            logging.config.dictConfig(log_file)
        print("logging configured")
        return logging.getLogger("echoflow")

    def log(self, msg, level, extra):
        self.logger.log(level=logging.DEBUG, msg=msg, extra=extra)
        self.logger.log(level=logging.ERROR, msg=msg, extra=extra)
        self.logger.log(level=logging.INFO, msg=msg, extra=extra)
        self.logger.log(level=logging.WARNING, msg=msg, extra=extra)


In [3]:
@flow(name="flow1")
def flow1():
    print(Singleton_Echoflow(log_file='../echoflow/config/logging_config.yaml'))
    print("logging done")
    flow2()

@flow(name="flow2")
def flow2():
    gea = Singleton_Echoflow.get_instance()
    gea.log('Demo Message', level="DEBUG", extra={"mod_name": "loggingDemo", "func_name": __name__})

In [4]:
flow1()

14:20:02.661 | INFO    | prefect.engine - Created flow run 'aquamarine-muskox' for flow 'flow1'

logging configured
logging done


14:20:03.007 | INFO    | Flow run 'aquamarine-muskox' - Created subflow run 'amphibian-jellyfish' for flow 'flow2'

/Users/soham/anaconda3/envs/echoflow/lib/python3.10/site-packages/watchtower/__init__.py:429: WatchtowerWarning: Received message after logging system shutdown
  warnings.warn("Received message after logging system shutdown", WatchtowerWarning)


14:20:03.094 | INFO    | Flow run 'amphibian-jellyfish' - Finished in state Completed()

14:20:03.121 | INFO    | Flow run 'aquamarine-muskox' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]

In [5]:
logging.getLogger("echoflow").error("Demo Debug Message", extra={"mod_name": "loggingDemo", "func_name": __name__})